## Import & Environment Setting

In [ ]:
import torch
from torch import autograd
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import json
import gc
import numpy as np
import datetime
from collections import Counter

from utils.dataset import DetectionFolder
from utils.model import YoloV3, YoloLoss
from utils.postprocess import PostProcessor


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float

## Load Config

In [ ]:
with open("./config/config.json", "r") as config_file:
    main_config = json.load(config_file)

try:
    model_config = main_config['model']
    loss_config = main_config['loss']
    train_config = main_config['train']
except NameError:
    assert False, ('Failed to load config file')
except KeyError:
    assert False, ('Failed to find key on config file')

In [ ]:
model_config['device'] = device
model_config['dtype'] = dtype
model_config['attrib_count'] = 5 + model_config['class_count']

loss_config['device'] = device
loss_config['dtype'] = dtype
loss_config['attrib_count'] = model_config['attrib_count']

train_config['device'] = device
train_config['dtype'] = dtype


In [ ]:
if train_config['use_tensorboard']:
    writer = SummaryWriter(log_dir=train_config['tensorboard_dir'])
else:
    writer = None

## Build

In [ ]:
model = YoloV3(model_config)
model.to(model_config['device'])

In [ ]:
loss_func = YoloLoss(loss_config)

In [ ]:
train_context = { }

train_context['dataset'] = DetectionFolder(train_config['train_list'], train_config['train_image'], train_config['train_label'])
train_context['dataloader'] = DataLoader(train_context['dataset'], batch_size = train_config['batch_size'], num_workers = 4)
train_context['val_dataset'] = DetectionFolder(train_config['val_list'], train_config['train_image'], train_config['train_label'])
train_context['val_dataloader'] = DataLoader(train_context['val_dataset'], batch_size = 1, num_workers = 4)

train_context['epoch'] = 0
train_context['last_checkpoint'] = 0
train_context['target_accuaracy'] = train_config['target_accuaracy']
train_context['target_recall'] = train_config['target_recall']
train_context['target_precision'] = train_config['target_precision']

train_context['lr'] = train_config['init_lr']
train_context['loss_window'] = []


loss_context = { }
loss_context['post_conf_threshold'] = loss_config['post_conf_threshold']
loss_context['post_iou_threshold'] = loss_config['post_iou_threshold']
loss_context['acc_iou_threshold'] = loss_config['acc_iou_threshold']
loss_context['acc_start_epoch'] = loss_config['acc_start_epoch']

In [ ]:

lr_func = lambda epoch: train_context['lr']
optimizer = torch.optim.Adam(model.parameters(), lr = train_context['lr'])
#train_context['optimizer'] = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lr_func, last_epoch = -1)

In [ ]:
def train(model, loss_func, optimizer, scheduler, train_context, train_config, loss_context, epochs, tb_writer=None):
    
    postProcessor = PostProcessor()
    
    for _ in range(0, epochs):
        # training step
        model.train()
        torch.autograd.set_detect_anomaly(train_config['use_anomaly_detection'])
        
        print('epoch : ', train_context['epoch'])
        print('    time : ', datetime.datetime.now().time())
        print('    lr : ', train_context['lr'])
        if train_config['use_tensorboard'] and tb_writer is not None:
            tb_writer.add_scalar('Step/Learning Rate', train_context['lr'], train_context['epoch'])
        
        losses = []
        for idx, batches in enumerate(train_context['dataloader']):
            image = batches['image'].to(train_config['device'], dtype = train_config['dtype'])
            labels = batches['label'].to(train_config['device'], dtype = train_config['dtype'])
            label_len = batches['label_len'].to(train_config['device'], dtype = torch.long)
            
            # forward
            out1, out2, out3 = model(image)
       
            # clear optimizer
            optimizer.zero_grad()
        
            # loss
            loss = loss_func(torch.cat((out1, out2, out3), 1), labels, label_len)
            losses.append(loss.item() / train_config['batch_size'])
            
            # backward
            loss.backward()
            optimizer.step()
            
            # cleanup
            del image, labels, label_len, loss
            del out1, out2, out3
            gc.collect()
            torch.cuda.empty_cache()
    
        # print loss
        avg_loss = np.mean(losses) if len(losses) is not 0 else 0
        train_context['loss_window'].append(avg_loss)
        print('    loss : ', avg_loss)
        if train_config['use_tensorboard'] and tb_writer is not None:
            tb_writer.add_scalar('Loss/Training Loss', avg_loss, train_context['epoch'])
            tb_writer.add_scalar('Step/Window Size', len(train_context['loss_window']), train_context['epoch'])
        

        # validate step
        if 'val_dataset' in train_context and train_context['val_dataset'] is not None:
            with torch.no_grad():
                model.eval()
                torch.autograd.set_detect_anomaly(False)
            
                losses = []
                if loss_context['acc_start_epoch'] < train_context['epoch']:
                    accs = Counter({})
                for idx, batches in enumerate(train_context['val_dataloader']):
                
                    image = batches['image'].to(train_config['device'], dtype = train_config['dtype'])
                    labels = batches['label'].to(train_config['device'], dtype = train_config['dtype'])
                    label_len = batches['label_len'].to(train_config['device'], dtype = torch.long)
            
                    out1, out2, out3 = model(image)
                    pred = torch.cat((out1, out2, out3), 1)
        
                    loss = loss_func(pred, labels, label_len)
                    losses.append(loss.item())
                
                    if loss_context['acc_start_epoch'] < train_context['epoch']:
                        prediction = {}
                        prediction['image'] = batches['image'].cpu().permute(0, 2, 3, 1).squeeze(0).numpy()
                        prediction['pred'] = pred.cpu().detach().squeeze(0).numpy()
                        prediction['label'] = batches['label'].cpu().squeeze(0).numpy()
                        prediction['label_len'] = batches['label_len'].cpu().squeeze(0).numpy()
                
                        bboxes = postProcessor.CUSTOM2(prediction['pred'], loss_context)
                        acc = postProcessor.calcAccuracyMap(prediction['label'], prediction['label_len'], bboxes, loss_context)
                        accs = accs + Counter(acc)
            
                    # cleanup
                    del image, labels, label_len, loss
                    if loss_context['acc_start_epoch'] < train_context['epoch']:
                        del prediction, bboxes, acc
                    del out1, out2, out3
                    gc.collect()
                    torch.cuda.empty_cache()
    
                # print validation loss
                avg_loss = np.mean(losses) if len(losses) is not 0 else 0
                print('    validation loss : ', avg_loss)
                if loss_context['acc_start_epoch'] < train_context['epoch']:
                    print('    accs : ', accs)
                if train_config['use_tensorboard'] and tb_writer is not None:
                    tb_writer.add_scalar('Loss/Validation Loss', avg_loss, train_context['epoch'])
                    
                    if loss_context['acc_start_epoch'] < train_context['epoch']:
                        tp = accs['true positive']
                        fn = accs['false negative']
                        fp = accs['false positive'] + accs['duplicate']
                        accuracy = tp / (tp + fn + fp)
                        recall = tp / (tp + fn)
                        precision = tp / (tp + fp)
                        print('    accuracy : ', accuracy)
                        print('    recall : ', recall)
                        print('    precision : ', precision)
                        tb_writer.add_scalar('Accuracy/Accuracy', accuracy, train_context['epoch'])
                        tb_writer.add_scalar('Accuracy/Recall', recall, train_context['epoch'])
                        tb_writer.add_scalar('Accuracy/Precision', precision, train_context['epoch'])
                        
                        if(accuracy >= train_context['target_accuaracy'] and 
                           recall >= train_context['target_recall'] and 
                           precision >= train_context['target_precision']):
                            # save model
                            torch.save(model, train_config['checkpoint_dir'] + 'model_r_' + str(train_context['epoch']) + '.dat')
                            
                
                
            
        # update learning rate & scheduler
        #window_len = len(train_context['loss_window'])
        #if (len(train_context['loss_window']) >= train_config['lr_window'] and
        #    np.mean(train_context['loss_window']) * train_config['lr_threshold'] <= np.mean(train_context['loss_window'][-2:])):
            
        #    print('    window size : ', len(train_context['loss_window']))
        #    print('    decrease lr to : ', train_context['lr'] * train_config['lr_decay'])
                
        #    train_context['lr'] = train_context['lr'] * train_config['lr_decay']
        #    train_context['loss_window'] = []
            
        #if len(train_context['loss_window']) > 2 * train_config['lr_window']:
        #    train_context['loss_window'] = train_context['loss_window'][(train_config['lr_window'] * 3) // 2:]
        if train_context['epoch'] > 80:
            train_context['lr'] = train_context['lr'] * 0.97
            train_context['loss_window'] = []
        
        scheduler.step()
        
        # update context
        train_context['dataset'].shuffle()
        train_context['epoch'] += 1
        
        # save model
        if (train_context['epoch'] > train_config['checkpoint_start'] 
            and train_context['epoch'] % train_config['checkpoint'] is 0):
            train_context['last_checkpoint'] = train_config['checkpoint']
            torch.save(model, train_config['checkpoint_dir'] + 'model_' + str(train_context['epoch']) + '.dat')
            

## Run

In [ ]:
train(model, loss_func, optimizer, scheduler, train_context, train_config, loss_context, train_config['epochs'], writer)
if writer is not None:
    writer.flush()
    writer.close()

## Temporary Code